# RPM vs RPR2 basal-organoid-derived allograft tumors

## Ireland et al 2024 BioRxiv

### Analysis for Fig. 2l,m and Extended Data Fig. 4h

In [ ]:
#071624_RPM_RPR2_Allo_Cluster
#Clustering RPM Allograft with RPR2 TBO for Figure 2 AI 
#Import relevant packages

import numpy as np
import pandas as pd
from matplotlib import rcParams
import os
import scanpy as sc

import matplotlib as mpl
import matplotlib.pyplot as plt

#For nice color schemes
import cmocean

#For barplots
import seaborn as sns

#Import scVI
import scvi
from scvi.model.utils import mde

scvi.settings.verbosity = 40



In [ ]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

In [ ]:
# Read 
os.chdir('/work/asi16')

## 1. Read in allograft data from RPM and RPR2 tumors

In [ ]:
# Read in RPR2 and RPM allografts
RPR2_allo=sc.read_10x_mtx('062024_RPR2_TBO_Allo_custom/outs/filtered_feature_bc_matrix', var_names='gene_symbols', cache=True)
RPM_allo=sc.read_10x_mtx('MedGenome_FASTQ_123123/RPM_TBO_Allo/042024_RPM_TBO_Allo_CustomCount/outs/filtered_feature_bc_matrix', var_names='gene_symbols', cache=True)
RPM_allo2=sc.read_10x_mtx('02_2024_TBO_Analyses/Old_Xeno/042024_custom_RPMTBO_FXeno_Old/outs/filtered_feature_bc_matrix', var_names='gene_symbols', cache=True)


## 2. Concatenate tumor data from RPM and RPR2 models

In [ ]:
# Concatenate RPR2 w RPM allo cells
adata= RPR2_allo.concatenate([RPM_allo, RPM_allo2], index_unique=None, join="outer")
adata.obs.groupby(["Genotype"]).apply(len)

## 3. Perform QC

In [ ]:
# Perform QC on RPR2_Allo 
#QC filtering
RPR2_allo.var['mito'] = RPR2_allo.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(RPR2_allo, qc_vars=['mito'], percent_top=None, log1p=False, inplace=True)

In [ ]:
#QC filtering RPR2 only scanpy flow
sc.pp.filter_cells(RPR2_allo, min_genes=200)

RPR2_allo.var['mito'] = RPR2_allo.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(RPR2_allo, qc_vars=['mito'], percent_top=None, log1p=False, inplace=True)

In [ ]:
sc.pl.violin(RPR2_allo, ['n_genes_by_counts', 'total_counts', 'pct_counts_mito'],
             jitter=0.4, multi_panel=True)
sc.pl.scatter(RPR2_allo, x='total_counts', y='pct_counts_mito')
sc.pl.scatter(RPR2_allo, x='total_counts', y='n_genes_by_counts')

In [ ]:
#Filter data by slicing anndata object
RPR2_allo = RPR2_allo[RPR2_allo.obs.n_genes_by_counts < 9000, :]
RPR2_allo = RPR2_allo[RPR2_allo.obs.n_genes_by_counts > 500, :]
RPR2_allo = RPR2_allo[RPR2_allo.obs.total_counts > 2000, :]
RPR2_allo = RPR2_allo[RPR2_allo.obs.pct_counts_mito < 15, :]

In [ ]:
# Add metadata to RPR2 sample
RPR2_allo.obs['Genotype'] = 'RPR2'
RPR2_allo.obs['Model'] = 'Allograft'
RPR2_allo.obs['Cre'] = 'Cre'
RPR2_allo.obs['UnID'] = 'RPR2_Allo'
RPR2_allo.obs['Batch'] = 'RPR2_Allo'


In [ ]:
adata.obs.groupby(["UnID"]).apply(len)

In [ ]:
#Prep for HVG and scvi
#log1p the data
adata.obs["log1p_total_counts"] = np.log1p(adata.obs["total_counts"])

In [ ]:
#Create layers
adata.layers["counts"] = adata.X.copy()
adata.layers['norm'] = adata.X.copy(); sc.pp.normalize_total(adata, target_sum=1e4, layer="norm")

In [ ]:
#HVG via Scanpy
#Note here that if you run with a batch_key with few cells, will get b'reciprocal condition number error
sc.pp.highly_variable_genes(
    adata,n_top_genes=10000,
    subset=False,
    layer="counts",
    flavor="seurat_v3",
    batch_key="Batch"
)

## 4. Set up model and train (scvi)

In [ ]:
scvi.model.SCVI.setup_anndata(
    adata,
    layer="counts",
    batch_key='Batch',
    continuous_covariate_keys=["pct_counts_mito"]
)

In [ ]:
model = scvi.model.SCVI(adata)

In [ ]:
model.train()

In [ ]:
#Fit model to data
#Get latent representation of model to apply to UMAP
latent = model.get_latent_representation()
adata.obsm["X_scVI_1.1"] = latent

## 5. Perform leiden clustering

In [ ]:
#Calculate neighbors using scVI model input
sc.pp.neighbors(adata, use_rep="X_scVI_1.1")
sc.tl.umap(adata, min_dist=0.5)

#Run leiden clustering based on neighbors
sc.tl.leiden(adata, key_added="leiden_scVI_1.1", resolution=1)

## 6. Visualize data and get marker genes to ID high-quality tumor clusters only

In [ ]:
#QC UMAPs
sc.pl.umap(
    adata,
    color=["n_genes_by_counts", "total_counts", "pct_counts_mito", "log1p_total_counts"],
    cmap="cubehelix_r",
    s=3,
    ncols=2,
)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
sc.pl.umap(adata, color="Genotype", cmap="cmo.matter", s=30, ax=ax, vmax="p99.99", frameon=False, save=False)
fig, ax = plt.subplots(figsize=(8, 6))
sc.pl.umap(adata, color="leiden_scVI_1.1", legend_loc="on data", legend_fontsize='large',ax=ax, s=30, frameon=False, save=False)
fig, ax = plt.subplots(figsize=(8, 6))
sc.pl.umap(adata, color="Cre", legend_loc="right margin", ax=ax, s=30, frameon=False, save=False, palette=['purple','orange'])
fig, ax = plt.subplots(figsize=(8, 6))
sc.pl.umap(adata, color="Model", legend_loc="right margin", ax=ax, s=30, frameon=False, save=False, palette=['#0aa6d8','#a1c299'])
fig, ax = plt.subplots(figsize=(8, 6))
sc.pl.umap(adata, color="UnID", legend_loc="right margin", ax=ax, s=30, frameon=False, save=False)


#Additional QC bar graphs
adata.obs['cluster'] = adata.obs["leiden_scVI_1.1"].copy()

#Plot Log1p total counts
fig, ax = plt.subplots(figsize=(20,6))
sns.boxenplot(data=adata.obs, x="cluster", y="log1p_total_counts", ax=ax)

#Plot Pct counts mito
fig, ax = plt.subplots(figsize=(20,6))
sns.boxenplot(data=adata.obs, x="cluster", y="pct_counts_mito", ax=ax)

In [ ]:
#feature plots
 # updated 3/4/21 #
more_types=["Ptprc", "Acta2","Pecam1",
            "Pdpn","Cav1","Cav2","Hopx","Timp3","Sema3f","Serpine1", #AT1
              "Abca3","Muc1","Sftpa1","Sftpb","Sftpd","Scd1", #AT2
              "Scgb1a1","Cyp2f2","Scgb3a2", "Scgb3a1","Lypd2",#Club
              "Muc5ac","Muc5b", # Goblet
              "Tubb4a","Foxa3","Foxj1","Rfx2","Rfx3","Trp73", #Ciliated
              'Krt5', 'Krt17','Krt15','Trp63','Id1','Icam1','Epas1','Aqp3','Sfn','Perp','Fxyd3','Sdc1','Gstm2','F3','Abi3bp','Adh7', # Basal
              'Bex2','Ascl1','Meis2','Hes6','Neurod1','Neurod4','Nhlh1','Nhlh2',
              'Hoxb5','Foxa2','Sox4','Rora','Isl1','Id4', 
              'Pou2f3','Trpm5','Ascl2','Ehf',
              'Lrmp','Gng13','Ltc4s','Alox5ap','Avil','Alox5','Atp2a3','Plk2', #tuft
              "Cftr","Ascl3", 'Stap1','Atp6v1c2','Pparg','Rasd1','Slc12a2', #ionocyte
              "Gja1","Nkx2-1","Epcam", # Lung lineage
              'Yap1','Wwtr1','Sox2','Cd44','Hes1', # Stem-like
             "Venus","fLuc", "Top2a","Mki67",
            'GFP.CDS','CellTag.UTR'] # Tumor markers

sc.pl.umap(
    adata,
    color=more_types,
    use_raw=False,
    legend_loc= "on data",
    color_map="cmo.dense",
    ncols=4,
    frameon=False,
    layer="norm",
    save=False
)

In [ ]:
# Find cluster markers for each leiden cluster to aid filtering
sc.tl.rank_genes_groups(adata, 'leiden_scVI_1.1', method='wilcoxon', layer='norm', use_raw=False)
pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(50)

In [ ]:
adata.write_h5ad("071624_adata_RPM_RPR2_TBOAllo.h5ad")

In [ ]:
adata

In [ ]:
# Remove non-tumor/low quality clusters
# Cluster 19 immune, Cluster 20 fibroblast, 
# Subset adata_2 which contains RPM organoids and allografts (Fig 3f) to just have RPM tumor cells 

bad_clust=['19','20']

#Filter out bad clusters
to_keep=(~adata.obs['leiden_scVI_1.1'].isin(bad_clust))

#Copy over to new anndata object
adata_2 = adata[to_keep].copy()

### From here, continue iterating through runs of scvi modeling until no clear low quality cell clusters or non-tumor cells are observed.
### Start back up at "set up and train scvi model" and run through subsetting out "bad clusters".
### Each time clusters are removed, model is run again to recluster.

## ITERATION 2

In [ ]:
#HVG via Scanpy
#Note here that if you run with a batch_key with few cells, will get b'reciprocal condition number error
sc.pp.highly_variable_genes(
    adata_2,n_top_genes=10000,
    subset=False,
    layer="counts",
    flavor="seurat_v3",
    batch_key="Batch"
)

In [ ]:
scvi.model.SCVI.setup_anndata(
    adata_2,
    layer="counts",
    batch_key='Batch',
    continuous_covariate_keys=["pct_counts_mito"]
)

In [ ]:
model = scvi.model.SCVI(adata_2)

In [ ]:
adata_2.obs.groupby(["UnID"]).apply(len)

In [ ]:
model.train()

In [ ]:
#Fit model to data
#Get latent representation of model to apply to UMAP
latent = model.get_latent_representation()

adata_2.obsm["X_scVI_1.2"] = latent

In [ ]:
#Calculate neighbors using scVI model input
sc.pp.neighbors(adata_2, use_rep="X_scVI_1.2")
sc.tl.umap(adata_2, min_dist=0.5)

#Run leiden clustering based on neighbors
sc.tl.leiden(adata_2, key_added="leiden_scVI_1.2", resolution=1)

In [ ]:
#QC UMAPs
sc.pl.umap(
    adata_2,
    color=["n_genes_by_counts", "total_counts", "pct_counts_mito", "log1p_total_counts"],
    cmap="cubehelix_r",
    s=3,
    ncols=2,
)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
sc.pl.umap(adata_2, color="Genotype", cmap="cmo.matter", s=30, ax=ax, vmax="p99.99", frameon=False, save=False)
fig, ax = plt.subplots(figsize=(8, 6))
sc.pl.umap(adata_2, color="leiden_scVI_1.1", legend_loc="on data", legend_fontsize='large',ax=ax, s=30, frameon=False, save=False)
fig, ax = plt.subplots(figsize=(8, 6))
sc.pl.umap(adata_2, color="Cre", legend_loc="right margin", ax=ax, s=30, frameon=False, save=False, palette=['purple','orange'])
fig, ax = plt.subplots(figsize=(8, 6))
sc.pl.umap(adata_2, color="Model", legend_loc="right margin", ax=ax, s=30, frameon=False, save=False, palette=['#0aa6d8','#a1c299'])
fig, ax = plt.subplots(figsize=(8, 6))
sc.pl.umap(adata_2, color="UnID", legend_loc="right margin", ax=ax, s=30, frameon=False, save=False)


#Additional QC bar graphs
adata_2.obs['cluster'] = adata_2.obs["leiden_scVI_1.2"].copy()

#Plot Log1p total counts
fig, ax = plt.subplots(figsize=(20,6))
sns.boxenplot(data=adata_2.obs, x="cluster", y="log1p_total_counts", ax=ax)

#Plot Pct counts mito
fig, ax = plt.subplots(figsize=(20,6))
sns.boxenplot(data=adata_2.obs, x="cluster", y="pct_counts_mito", ax=ax)

In [ ]:
#feature plots
 # updated 3/4/21 #
more_types=["Ptprc", "Acta2","Pecam1",
            "Pdpn","Cav1","Cav2","Hopx","Timp3","Sema3f","Serpine1", #AT1
              "Abca3","Muc1","Sftpa1","Sftpb","Sftpd","Scd1", #AT2
              "Scgb1a1","Cyp2f2","Scgb3a2", "Scgb3a1","Lypd2",#Club
              "Muc5ac","Muc5b", # Goblet
              "Tubb4a","Foxa3","Foxj1","Rfx2","Rfx3","Trp73", #Ciliated
              'Krt5', 'Krt17','Krt15','Trp63','Id1','Icam1','Epas1','Aqp3','Sfn','Perp','Fxyd3','Sdc1','Gstm2','F3','Abi3bp','Adh7', # Basal
              'Bex2','Ascl1','Meis2','Hes6','Neurod1','Neurod4','Nhlh1','Nhlh2',
              'Hoxb5','Foxa2','Sox4','Rora','Isl1','Id4', 
              'Pou2f3','Trpm5','Ascl2','Ehf',
              'Lrmp','Gng13','Ltc4s','Alox5ap','Avil','Alox5','Atp2a3','Plk2', #tuft
              "Cftr","Ascl3", 'Stap1','Atp6v1c2','Pparg','Rasd1','Slc12a2', #ionocyte
              "Gja1","Nkx2-1","Epcam", # Lung lineage
              'Yap1','Wwtr1','Sox2','Cd44','Hes1', # Stem-like
             "Venus","fLuc", "Top2a","Mki67",
            'GFP.CDS','CellTag.UTR'] # Tumor markers

sc.pl.umap(
    adata_2,
    color=more_types,
    use_raw=False,
    legend_loc= "on data",
    color_map="cmo.dense",
    ncols=4,
    frameon=False,
    layer="norm",
    save=False
)

In [ ]:
# Find cluster markers for each leiden cluster to aid filtering
sc.tl.rank_genes_groups(adata_2, 'leiden_scVI_1.2', method='wilcoxon', layer='norm', use_raw=False)
pd.DataFrame(adata_2.uns['rank_genes_groups']['names']).head(50)

In [ ]:
adata_2.write_h5ad("071624_adata_2_RPM_RPR2_TBOAllo.h5ad")

In [ ]:
adata_2=sc.read_h5ad("071624_adata_2_RPM_RPR2_TBOAllo.h5ad")

In [ ]:
# Remove non-tumor/low quality clusters
# Cluster 15 seems like normal cells.. alveolar/secretory/basal

bad_clust=['15']

#Filter out bad clusters
to_keep=(~adata_2.obs['leiden_scVI_1.2'].isin(bad_clust))

#Copy over to new anndata object
adata_3 = adata_2[to_keep].copy()

## ITERATION 3 (Final iteration for Fig 2l,m and Ext Data Fig 4h)

In [ ]:
#HVG via Scanpy
#Note here that if you run with a batch_key with few cells, will get b'reciprocal condition number error
sc.pp.highly_variable_genes(
    adata_3,n_top_genes=10000,
    subset=False,
    layer="counts",
    flavor="seurat_v3",
    batch_key="Batch"
)

In [ ]:
scvi.model.SCVI.setup_anndata(
    adata_3,
    layer="counts",
    batch_key='Batch',
    continuous_covariate_keys=["pct_counts_mito"]
)

In [ ]:
model = scvi.model.SCVI(adata_3)

In [ ]:
adata_3.obs.groupby(["UnID"]).apply(len)

In [ ]:
model.train()

In [ ]:
#Fit model to data
#Get latent representation of model to apply to UMAP
latent = model.get_latent_representation()

adata_3.obsm["X_scVI_1.3"] = latent

In [ ]:
#Calculate neighbors using scVI model input
sc.pp.neighbors(adata_3, use_rep="X_scVI_1.3")
sc.tl.umap(adata_3, min_dist=0.5)

#Run leiden clustering based on neighbors
sc.tl.leiden(adata_3, key_added="leiden_scVI_1.3", resolution=0.5)

In [ ]:
#QC UMAPs
sc.pl.umap(
    adata_3,
    color=["n_genes_by_counts", "total_counts", "pct_counts_mito", "log1p_total_counts"],
    cmap="cubehelix_r",
    s=3,
    ncols=2,
)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
sc.pl.umap(adata_3, color="Genotype", cmap="cmo.matter", s=30, ax=ax, vmax="p99.99", frameon=False, save=False,palette=["#22e2d9","#206e71"])
fig, ax = plt.subplots(figsize=(8, 6))
sc.pl.umap(adata_3, color="leiden_scVI_1.3", legend_loc="on data", legend_fontsize='large',ax=ax, s=30, frameon=False, save=False)
fig, ax = plt.subplots(figsize=(8, 6))
sc.pl.umap(adata_3, color="Cre", legend_loc="right margin", ax=ax, s=30, frameon=False, save=False, palette=['purple','orange'])
fig, ax = plt.subplots(figsize=(8, 6))
sc.pl.umap(adata_3, color="Model", legend_loc="right margin", ax=ax, s=30, frameon=False, save=False, palette=['#0aa6d8','#a1c299'])
fig, ax = plt.subplots(figsize=(8, 6))
sc.pl.umap(adata_3, color="UnID", legend_loc="right margin", ax=ax, s=30, frameon=False, save=False)


#Additional QC bar graphs
adata_3.obs['cluster'] = adata_3.obs["leiden_scVI_1.3"].copy()

#Plot Log1p total counts
fig, ax = plt.subplots(figsize=(20,6))
sns.boxenplot(data=adata_3.obs, x="cluster", y="log1p_total_counts", ax=ax)

#Plot Pct counts mito
fig, ax = plt.subplots(figsize=(20,6))
sns.boxenplot(data=adata_3.obs, x="cluster", y="pct_counts_mito", ax=ax)

In [ ]:
#feature plots
 # updated 3/4/21 #
more_types=["Ptprc", "Acta2","Pecam1",
            "Pdpn","Cav1","Cav2","Hopx","Timp3","Sema3f","Serpine1", #AT1
              "Abca3","Muc1","Sftpa1","Sftpb","Sftpd","Scd1", #AT2
              "Scgb1a1","Cyp2f2","Scgb3a2", "Scgb3a1","Lypd2",#Club
              "Muc5ac","Muc5b", # Goblet
              "Tubb4a","Foxa3","Foxj1","Rfx2","Rfx3","Trp73", #Ciliated
              'Krt5', 'Krt17','Krt15','Trp63','Id1','Icam1','Epas1','Aqp3','Sfn','Perp','Fxyd3','Sdc1','Gstm2','F3','Abi3bp','Adh7', # Basal
              'Bex2','Ascl1','Meis2','Hes6','Neurod1','Neurod4','Nhlh1','Nhlh2',
              'Hoxb5','Foxa2','Sox4','Rora','Isl1','Id4', 
              'Pou2f3','Trpm5','Ascl2','Ehf',
              'Lrmp','Gng13','Ltc4s','Alox5ap','Avil','Alox5','Atp2a3','Plk2', #tuft
              "Cftr","Ascl3", 'Stap1','Atp6v1c2','Pparg','Rasd1','Slc12a2', #ionocyte
              "Gja1","Nkx2-1","Epcam", # Lung lineage
              'Yap1','Wwtr1','Sox2','Cd44','Hes1', # Stem-like
             "Venus","fLuc", "Top2a","Mki67",
            'GFP.CDS','CellTag.UTR'] # Tumor markers

sc.pl.umap(
    adata_3,
    color=more_types,
    use_raw=False,
    legend_loc= "on data",
    color_map="cmo.dense",
    ncols=4,
    frameon=False,
    layer="norm",
    save=False
)

In [ ]:
adata_3.obs_names_make_unique()

In [ ]:
# Save and export for R/Seurat and signature assessment
adata_3.write_h5ad("090824_adata_3_RPM_RPR2_TBOAllo.h5ad")

In [ ]:
adata_3=sc.read_h5ad("090824_adata_3_RPM_RPR2_TBOAllo.h5ad")

In [ ]:
#Generate signatures from these two tumors
sc.pp.normalize_total(adata_3)
sc.pp.log1p(adata_3)
sc.tl.rank_genes_groups(adata_3, 'leiden_scVI_1.3', method='t-test')


# Extract top 500 marker genes for leiden clusters from data 
sc.tl.rank_genes_groups(adata_3,'leiden_scVI_1.3', method='wilcoxon', n_genes=500)

result = adata_3.uns['rank_genes_groups']
groups = result['names'].dtype.names
markergenes=pd.DataFrame(
    {group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names', 'pvals']})

markergenes.to_csv('/hpc/home/asi16/RPMvRPR2_Allo_Leiden_scRNAseq_100724.csv' )

In [ ]:
# Feature plots
# updated 3/4/21
more_types=["Ascl1","Neurod1","Pou2f3"] # Tumor markers

sc.pl.umap(
    adata_3,
    color=more_types,
    use_raw=False,
    legend_loc= "on data",
    color_map="cmo.dense",
    ncols=3,s=40,
    frameon=False,
    layer="norm",
    save=False
)

In [ ]:
# Feature plots
# updated 3/4/21

more_types=["Myc","Mycl"]

sc.pl.umap(
    adata_3,
    color=more_types,
    use_raw=False,
    legend_loc= "on data",
    color_map="cmo.dense",
    ncols=2,s=60,
    frameon=False,
    layer="norm",
    save=False
)